## Homework

[ Source : https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md ]

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [40]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [2]:
taxi_jan23 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
taxi_jan23.shape

(3066766, 19)

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [5]:
taxi_jan23.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [12]:
taxi_jan23['duration'] = taxi_jan23.apply(lambda x : (x['tpep_dropoff_datetime']-x['tpep_pickup_datetime']).total_seconds()/60, axis=1)
taxi_jan23.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [13]:
taxi_jan23['duration'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [23]:
taxi_jan23_le_1hr = taxi_jan23[(taxi_jan23.duration >= 1) & (taxi_jan23.duration <= 60)].copy()
print("Total Rows: ", taxi_jan23.shape[0], "\nRows with duration le 1hr.: ", taxi_jan23_le_1hr.shape[0], "\nRows with no outliers: ", taxi_jan23_le_1hr.shape[0]/taxi_jan23.shape[0]*100)

Total Rows:  3066766 
Rows with duration le 1hr.:  3009173 
Rows with no outliers:  98.1220282212598


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will
  label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [24]:
categorical = ['PULocationID', 'DOLocationID']
taxi_jan23_le_1hr[categorical] = taxi_jan23_le_1hr[categorical].astype(str)

dv = DictVectorizer()

train_dicts = taxi_jan23_le_1hr[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [34]:
len(dv.feature_names_), X_train

(515,
 <3009173x515 sparse matrix of type '<class 'numpy.float64'>'
 	with 6018346 stored elements in Compressed Sparse Row format>)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [43]:
target = 'duration'
y_train = taxi_jan23_le_1hr[target].values

lrmodel = LinearRegression()
lrmodel.fit(X_train, y_train)

y_pred = lrmodel.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.649261929771859

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [44]:
target = 'duration'
categorical = ['PULocationID', 'DOLocationID']

taxi_feb23 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

taxi_feb23['duration'] = taxi_feb23.apply(lambda x : (x['tpep_dropoff_datetime']-x['tpep_pickup_datetime']).total_seconds()/60, axis=1)
taxi_feb23_le_1hr = taxi_feb23[(taxi_feb23.duration >= 1) & (taxi_feb23.duration <= 60)].copy()

taxi_feb23_le_1hr[categorical] = taxi_feb23_le_1hr[categorical].astype(str)
val_dicts = taxi_feb23_le_1hr[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = taxi_feb23_le_1hr[target].values

y_pred = lrmodel.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.811818933419717